Ссылка на [контест](https://www.kaggle.com/c/ozon-masters-ds-contest-2021/overview)  

### Используемая модель  
Для решения данной задачи была использована модель **градиентного бустинга** библиотеки **CatBoost**.  
- Градиентный бустинг - потому что это мощный и универсальный алгоритм машинного обучения в силу своей адаптивной техники построения композиции (грех не попробовать).  
- CatBoost - потому что это зарекомендовавшая себя библиотека от Яндекса, реализующая градиентный бустинг над решающими деревьями, которая в сравнительном тестировании выигрывает у многих аналогов [источник](https://catboost.ai/#benchmark).

### Используемые подходы  

- Кодирование категориальных признаков с помощью **One-Hot Encoding** (режим игры, типы юнитов). Из плюсов: результат является двоичным, а не порядковым, модель лучше воспринимает влияние признаков в таком виде. Из минусов: это увеличивает размерность данных (в данном случае существенно, 209 (7 + 202) новых признака)
- Удаление из исходного датасета признаков, которые незначимы (в данном случае это признаки id, X1, X3). Их незначимость проверялась экспериментально (их наличие увеличивает ошибку). Но и здравый смысл подсказывает, что они не несут в себе никакой информации для определения победителя игры
- Подбор параметров модели (iterations, learning_rate). Learning_rate (коэффициент скорости обучения) = 0.5, т.к. это значение позволяет проходить обучению достаточно быстрыми шагами без ущерба для точности. Iterations (количество итераций) = 1000, т.к. при дальшейших итерациях значение ошибки начинает скакать и увеличиваться.

### Проверялось, но не используется в конечном варианте, т.к. не дало результата  
  
- Замена признаков X2 и X4 (рейтингов игроков) их разностью. Т.е. гипотеза в том, что значима разница рейтингов игроков, а не сами их величины.
- Также были испробованы другие модели машинного обучения: линейная регрессия и градиентный бустинг библиотеки sklearn, подбор оптимальных параметров, количества деревьев, но CatBoost победил в этой борьбе)

### Результат  

27/122 в приватном и 31/122 а публичном лидерборде 

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [2]:
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

Кодирую категориальные признаки (режим игры и типы юнитов) с помощью one-hot encoding

In [3]:
ohe = OneHotEncoder(handle_unknown='ignore', sparse = False)
new_ohe_features = ohe.fit_transform(data_train.X0.values.reshape(-1,1))
mode_values = np.sort(data_train.X0.unique())
tmp = pd.DataFrame(new_ohe_features, columns=['mode=' + str(i) for i in mode_values])
data_train_ohe = pd.concat([data_train, tmp], axis = 1)

In [4]:
units1 = ['X'+str(i) for i in range(5, 13)]
units2 = ['X'+str(i) for i in range(13, 21)]
units = units1 + units2
for col in units1:
    if col == units1[0]:
        df_indic1 = pd.get_dummies(data_train_ohe[col], prefix= 'unit1_')
    else:
        df_indic_units1 = pd.get_dummies(data_train_ohe[col], prefix= 'unit1_')
        df_indic1 = df_indic1 + df_indic_units1
for col in units2:
    if col == units2[0]:
        df_indic2 = pd.get_dummies(data_train_ohe[col], prefix= 'unit2_')
    else:
        df_indic_units2 = pd.get_dummies(data_train_ohe[col], prefix= 'unit2_')
        df_indic2 = df_indic2 + df_indic_units2 
        
data_train_ohe_total = pd.concat([data_train_ohe, df_indic1, df_indic2], axis = 1)

Удаляю ненужные признаки из обучающего датасета: целевую переменную, те признаки, которые перекодировала (в новых колонках) и те, которые незначимы (по моему мнению и по результату моих экспериментов с ними)

In [5]:
data_train_ohe_total = data_train_ohe_total.drop(units, axis=1)
data_train_ohe_total = data_train_ohe_total.drop(['id', 'X0', 'X1', 'X3', 'target'] , axis=1)

In [6]:
for i in data_train_ohe_total.columns:
    data_train_ohe_total[i] = data_train_ohe_total[i].astype(np.int32)

In [7]:
cat_features = [i for i in range(len(data_train_ohe_total.columns))]
cat_features.remove(0) # порядковый (уровень 1-го игрока)
cat_features.remove(1) # порядковый (уровень 2-го игрока)
cat_features.remove(2) # порядковый (время игры)

y = data_train.target
X = data_train_ohe_total

X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.8, random_state=1234)

model = CatBoostClassifier(
    iterations=1000,
    random_seed=63,
    learning_rate=0.5)
model.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_validation, y_validation),
    verbose = 50,
    plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6712331	test: 0.6709367	best: 0.6709367 (0)	total: 1.26s	remaining: 20m 59s
50:	learn: 0.6515014	test: 0.6513872	best: 0.6513872 (50)	total: 45.7s	remaining: 14m 9s
100:	learn: 0.6464195	test: 0.6468096	best: 0.6468096 (100)	total: 1m 28s	remaining: 13m 6s
150:	learn: 0.6432673	test: 0.6442510	best: 0.6442510 (150)	total: 2m 6s	remaining: 11m 49s
200:	learn: 0.6410518	test: 0.6427247	best: 0.6427247 (200)	total: 2m 43s	remaining: 10m 48s
250:	learn: 0.6392088	test: 0.6415055	best: 0.6415055 (250)	total: 3m 29s	remaining: 10m 25s
300:	learn: 0.6377057	test: 0.6406369	best: 0.6406369 (300)	total: 4m 13s	remaining: 9m 49s
350:	learn: 0.6363729	test: 0.6399717	best: 0.6399717 (350)	total: 4m 51s	remaining: 8m 58s
400:	learn: 0.6351559	test: 0.6393821	best: 0.6393821 (400)	total: 5m 29s	remaining: 8m 12s
450:	learn: 0.6340824	test: 0.6390260	best: 0.6390260 (450)	total: 6m 10s	remaining: 7m 30s
500:	learn: 0.6330436	test: 0.6386209	best: 0.6386209 (500)	total: 6m 48s	remaining: 

In [8]:
X_test = data_test

ohe = OneHotEncoder(handle_unknown='ignore', sparse = False)
new_ohe_features = ohe.fit_transform(X_test.X0.values.reshape(-1,1))
mode_values = np.sort(X_test.X0.unique())
tmp = pd.DataFrame(new_ohe_features, columns=['mode=' + str(i) for i in mode_values])
data_test_ohe = pd.concat([X_test, tmp], axis = 1)

for col in units1:
    if col == units1[0]:
        df_indic1 = pd.get_dummies(data_test_ohe[col], prefix= 'unit1_')
    else:
        df_indic_units1 = pd.get_dummies(data_test_ohe[col], prefix= 'unit1_')
        df_indic1 = df_indic1 + df_indic_units1
for col in units2:
    if col == units2[0]:
        df_indic2 = pd.get_dummies(data_test_ohe[col], prefix= 'unit2_')
    else:
        df_indic_units2 = pd.get_dummies(data_test_ohe[col], prefix= 'unit2_')
        df_indic2 = df_indic2 + df_indic_units2 
    
data_test_ohe_total = pd.concat([data_test_ohe, df_indic1, df_indic2], axis = 1)

data_test_ohe_total = data_test_ohe_total.drop(units, axis=1)
data_test_ohe_total = data_test_ohe_total.drop(['id', 'X0', 'X1', 'X3'] , axis=1)

for i in data_test_ohe_total.columns:
    data_test_ohe_total[i] = data_test_ohe_total[i].astype(np.int32)

cat_features = [i for i in range(len(data_test_ohe_total.columns))]
cat_features.remove(0) # порядковый
cat_features.remove(1) # порядковый
cat_features.remove(2) # порядковый

X_test = data_test_ohe_total
test_pool = Pool(data=X_test, cat_features=cat_features)
contest_predictions = model.predict_proba(test_pool)

f = open('submit.csv', 'w')
f.write('Id,target\n')
for idx in range(len(contest_predictions)):
    line = str(idx) + ',' + str(round(contest_predictions[idx][1],3)) + '\n'
    f.write(line)
f.close()